In [15]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
from sklearn import model_selection

In [16]:
df=pd.read_csv('merc_label.csv')
df_2=df.copy()
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024 entries, 0 to 1023
Data columns (total 9 columns):
class_                1024 non-null int64
variant_              1024 non-null int64
km                    1024 non-null int64
engine_capacity_cc    1024 non-null int64
fuel                  1024 non-null int64
gear                  1024 non-null int64
city                  1024 non-null int64
price_tl              1024 non-null int64
age                   1024 non-null int64
dtypes: int64(9)
memory usage: 72.1 KB


In [17]:
df_2.head()

,class_,variant_,km,engine_capacity_cc,fuel,gear,city,price_tl,age
0,0,0,41000,1,0,2,1,325950,6
1,0,0,102000,1,0,0,1,242000,5
2,0,12,0,1,0,2,1,440000,2
3,0,0,160000,1,0,2,0,239250,8
4,0,0,91000,1,0,2,1,303750,6


In [18]:
X = df.drop(["price_tl"], axis = 1)
Y = df["price_tl"]

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 144)

In [20]:
params = {"colsample_bytree":[0.4,0.5,0.6],
         "learning_rate":[0.01,0.02,0.09],
         "max_depth":[2,3,4,5,6],
         "n_estimators":[100,200,500,2000]}

In [21]:
xgb = XGBRegressor()

In [22]:
grid = GridSearchCV(xgb, params, cv = 10, n_jobs = -1, verbose = 2)

In [24]:
grid.fit(X_train, Y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   39.8s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:  3.6min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
       colsample_bynode=None, colsample_bytree=None, gamma=None,
       gpu_id=None, importance_type='gain', interaction_constraints=None,
       learning_rate=None, max_delta_step=None, max_depth=None,
       min_child_we..._pos_weight=None, subsample=None,
       tree_method=None, validate_parameters=None, verbosity=None),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'colsample_bytree': [0.4, 0.5, 0.6], 'learning_rate': [0.01, 0.02, 0.09], 'max_depth': [2, 3, 4, 5, 6], 'n_estimators': [100, 200, 500, 2000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [25]:
grid.best_params_

{'colsample_bytree': 0.5,
 'learning_rate': 0.02,
 'max_depth': 3,
 'n_estimators': 2000}

In [26]:
xgb1 = XGBRegressor(colsample_bytree = 0.5, learning_rate = 0.02, max_depth = 3, n_estimators = 2000)

In [28]:
model_xgb = xgb1.fit(X_train, Y_train)

In [29]:
model_xgb.predict(X_test)[15:20]

array([215396.67, 203489.08, 428759.9 , 532454.75, 391535.53],
      dtype=float32)

In [30]:
Y_test[15:20]

703    189000
121    198000
160    457500
737    495000
701    459900
Name: price_tl, dtype: int64

In [31]:
model_xgb.score(X_test, Y_test)

0.9558197675181905

In [32]:
model_xgb.score(X_train, Y_train)

0.9789842513710076

In [33]:
np.sqrt(-1*(cross_val_score(model_xgb, X_test, Y_test, cv=10, scoring='neg_mean_squared_error'))).mean()

71178.69893701086

In [35]:
importance = pd.DataFrame({"Importance": model_xgb.feature_importances_},
                         index=X_train.columns)
importance

,Importance
class_,0.239165
variant_,0.113917
km,0.093141
engine_capacity_cc,0.101326
fuel,0.032330
gear,0.087007
city,0.002973
age,0.330140
